## Lectura de datos

In [95]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

type1_encoder = LabelEncoder()
type2_encoder = LabelEncoder()
genre_enconder = LabelEncoder()

In [119]:
df_active_shows = pd.read_csv('/content/drive/MyDrive/Lulobank/model/Shows_Broadcasting_Information.csv',index_col=0)

df_shows = pd.read_csv('/content/drive/MyDrive/Lulobank/model/Shows_General_Information.csv',index_col=0)

df_schedules = pd.read_csv('/content/drive/MyDrive/Lulobank/model/Schedules_Shows.csv',index_col=0)

df_webChannel = pd.read_csv('/content/drive/MyDrive/Lulobank/model/WebChannel_Shows.csv',index_col=0)

df_external_links = pd.read_csv('/content/drive/MyDrive/Lulobank/model/External_links_Shows.csv',index_col=0)

df_genres = pd.read_csv('/content/drive/MyDrive/Lulobank/model/genres_shows.csv',index_col=0)

In [109]:
df_active_shows['program_type_id'] = type1_encoder.fit_transform(df_active_shows['type'])
df_shows['show_type_id'] = type2_encoder.fit_transform(df_shows['type'])

## Punto A : averageRuntime por tipo (type) por mes.

In [111]:
df_avgtime = pd.merge(df_active_shows,df_shows[['averageRuntime','episode_ids','show_type_id']], how = 'left', on = 'episode_ids')

In [112]:
df_type1 = df_avgtime.groupby(['program_type_id']).mean()['averageRuntime'].reset_index()
df_type2 = df_avgtime.groupby(['show_type_id']).mean()['averageRuntime'].reset_index()

In [113]:
df_type1['program_type'] = type1_encoder.inverse_transform(df_type1[['program_type_id']])
df_type1

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,program_type_id,averageRuntime,program_type
0,0,43.769231,insignificant_special
1,1,36.192446,regular
2,2,25.727273,significant_special


In [114]:
df_type2['show_type'] = type2_encoder.inverse_transform(df_type2[['show_type_id']].astype(int))
df_type2

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,show_type_id,averageRuntime,show_type
0,0.0,13.049383,Animation
1,1.0,204.000000,Award Show
2,2.0,33.414634,Documentary
3,3.0,63.571429,Game Show
4,4.0,74.800000,News
5,5.0,45.000000,Panel Show
6,6.0,38.509804,Reality
7,7.0,33.277778,Scripted
8,8.0,111.650000,Sports
9,9.0,41.459016,Talk Show


## Punto B :Cantidad de series emitidas por género por mes.

In [124]:
df_active_show = pd.merge(df_active_shows,df_shows[['episode_ids','shows_id']], how = 'left', on = 'episode_ids')
df_genres_show = pd.merge(df_active_show['shows_id'] ,df_genres, how = 'left', on = 'shows_id')

In [125]:
df_genres_counted = df_genres_show.sum().reset_index()[1:-1]
df_genres_counted.columns = ['genre','quantity']
df_genres_counted.sort_values(by ='quantity', ascending =False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


,genre,quantity
6,Comedy,179.0
9,Drama,169.0
20,Romance,89.0
5,Children,40.0
7,Crime,40.0
1,Action,38.0
3,Adventure,35.0
11,Fantasy,35.0
18,Mystery,31.0
4,Anime,30.0


## Punto C: Porcentaje por país, con respecto al total de series emitidas por mes.

In [149]:
df_channels_show = pd.merge(df_active_show[['rating','shows_id']] ,df_webChannel[['shows_id','country.name']], how = 'left', on = 'shows_id')
total_shows = len(df_channels_show)

In [150]:
df_channels_show['country.name'] = df_channels_show['country.name'].fillna('Not identify')
country_counted = df_channels_show.groupby('country.name').count()['shows_id']/total_shows
country_counted = country_counted.reset_index().sort_values(by = 'shows_id', ascending = False)
country_counted

,country.name,shows_id
21,Not identify,0.110049
32,United States,0.021746
5,China,0.016145
20,Norway,0.011862
25,Russian Federation,0.009226
17,"Korea, Republic of",0.007578
31,United Kingdom,0.003954
30,Turkey,0.002965
13,India,0.002306
10,Germany,0.002306


## Punto D : Rating promedio de series por país  por mes.

In [222]:
df_country = pd.merge(df_channels_show,df_genres.iloc[:,:-2], how = 'left', on = 'shows_id')
df_country_clean = df_country.groupby('country.name').mean()['rating'].reset_index()
df_country_clean.columns= ['country','avgrating']

# Exportar base de datos

In [223]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///LuloBankTestETLs.db', echo=True)
sqlite_connection = engine.connect()

In [224]:
df_type1.to_sql("AverageTypeProgram", sqlite_connection, if_exists='fail')

2022-06-27 23:43:32,872 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("AverageTypeProgram")
2022-06-27 23:43:32,875 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 23:43:32,879 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("AverageTypeProgram")
2022-06-27 23:43:32,883 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 23:43:32,887 INFO sqlalchemy.engine.Engine 
CREATE TABLE "AverageTypeProgram" (
	"index" BIGINT, 
	program_type_id BIGINT, 
	"averageRuntime" FLOAT, 
	program_type TEXT
)


2022-06-27 23:43:32,890 INFO sqlalchemy.engine.Engine [no key 0.00226s] ()
2022-06-27 23:43:32,904 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 23:43:32,906 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_AverageTypeProgram_index" ON "AverageTypeProgram" ("index")
2022-06-27 23:43:32,907 INFO sqlalchemy.engine.Engine [no key 0.00166s] ()
2022-06-27 23:43:32,923 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 23:43:32,927 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-27

In [225]:
df_type2.to_sql("AverageTypeShow", sqlite_connection, if_exists='fail')

2022-06-27 23:43:32,966 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("AverageTypeShow")
2022-06-27 23:43:32,971 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 23:43:32,974 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("AverageTypeShow")
2022-06-27 23:43:32,976 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 23:43:32,981 INFO sqlalchemy.engine.Engine 
CREATE TABLE "AverageTypeShow" (
	"index" BIGINT, 
	show_type_id FLOAT, 
	"averageRuntime" FLOAT, 
	show_type TEXT
)


2022-06-27 23:43:32,984 INFO sqlalchemy.engine.Engine [no key 0.00325s] ()
2022-06-27 23:43:33,000 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 23:43:33,002 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_AverageTypeShow_index" ON "AverageTypeShow" ("index")
2022-06-27 23:43:33,004 INFO sqlalchemy.engine.Engine [no key 0.00191s] ()
2022-06-27 23:43:33,015 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 23:43:33,019 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-27 23:43:33,022 INFO sql

In [226]:
df_genres_counted.to_sql("GenresCounted", sqlite_connection, if_exists='fail')

2022-06-27 23:43:33,049 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("GenresCounted")
2022-06-27 23:43:33,057 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 23:43:33,059 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("GenresCounted")
2022-06-27 23:43:33,061 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 23:43:33,065 INFO sqlalchemy.engine.Engine 
CREATE TABLE "GenresCounted" (
	"index" BIGINT, 
	genre TEXT, 
	quantity FLOAT
)


2022-06-27 23:43:33,067 INFO sqlalchemy.engine.Engine [no key 0.00187s] ()
2022-06-27 23:43:33,079 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 23:43:33,082 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_GenresCounted_index" ON "GenresCounted" ("index")
2022-06-27 23:43:33,092 INFO sqlalchemy.engine.Engine [no key 0.00981s] ()
2022-06-27 23:43:33,103 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 23:43:33,105 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-27 23:43:33,107 INFO sqlalchemy.engine.Engine INSERT INTO "GenresCou

In [227]:
country_counted.to_sql("CountryPercentage", sqlite_connection, if_exists='fail')

2022-06-27 23:43:33,143 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CountryPercentage")
2022-06-27 23:43:33,146 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 23:43:33,149 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CountryPercentage")
2022-06-27 23:43:33,153 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 23:43:33,158 INFO sqlalchemy.engine.Engine 
CREATE TABLE "CountryPercentage" (
	"index" BIGINT, 
	"country.name" TEXT, 
	shows_id FLOAT
)


2022-06-27 23:43:33,161 INFO sqlalchemy.engine.Engine [no key 0.00333s] ()
2022-06-27 23:43:33,176 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 23:43:33,177 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_CountryPercentage_index" ON "CountryPercentage" ("index")
2022-06-27 23:43:33,179 INFO sqlalchemy.engine.Engine [no key 0.00141s] ()
2022-06-27 23:43:33,190 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 23:43:33,194 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-27 23:43:33,197 INFO sqlalchemy.engine.

In [228]:
df_country_clean.to_sql("CountryRating", sqlite_connection, if_exists='fail')

2022-06-27 23:43:33,224 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CountryRating")
2022-06-27 23:43:33,226 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 23:43:33,240 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("CountryRating")
2022-06-27 23:43:33,246 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-06-27 23:43:33,249 INFO sqlalchemy.engine.Engine 
CREATE TABLE "CountryRating" (
	"index" BIGINT, 
	country TEXT, 
	avgrating FLOAT
)


2022-06-27 23:43:33,252 INFO sqlalchemy.engine.Engine [no key 0.00239s] ()
2022-06-27 23:43:33,263 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 23:43:33,265 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_CountryRating_index" ON "CountryRating" ("index")
2022-06-27 23:43:33,266 INFO sqlalchemy.engine.Engine [no key 0.00131s] ()
2022-06-27 23:43:33,277 INFO sqlalchemy.engine.Engine COMMIT
2022-06-27 23:43:33,280 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-06-27 23:43:33,286 INFO sqlalchemy.engine.Engine INSERT INTO "Countr